# PRMT-1984 [HYPOTHESIS] A high proportion of pending transfers are caused by a small group of practices

### Hypothesis
We believe that a large proportion of pending transfers are caused by a small group of practices receiving, regardless of the sending supplier. 

We will know this to be true when we can see that the same practices causing a high number of EMIS-EMIS pending transfers are the same as those causing TPP-EMIS or Vision-EMIS, (and the same would occur for each supplier as a receiver)

In [168]:
import pandas as pd
import numpy as np

transfer_file_location = "s3://prm-gp2gp-data-sandbox-dev/transfers-duplicates-hypothesis/"
transfer_files = [
    "9-2020-transfers.parquet",
    "10-2020-transfers.parquet",
    "11-2020-transfers.parquet",
    "12-2020-transfers.parquet",
    "1-2021-transfers.parquet",
    "2-2021-transfers.parquet"
]
transfer_input_files = [transfer_file_location + f for f in transfer_files]
transfers_raw = pd.concat((
    pd.read_parquet(f)
    for f in transfer_input_files
))
# This is only needed when using transfers-duplicates-hypothesis datasets
transfers_raw = transfers_raw.drop(["sending_supplier", "requesting_supplier"], axis=1)


# Given the findings in PRMT-1742 - many duplicate EHR errors are misclassified, the below reclassifies the relevant data
successful_transfers_bool = transfers_raw['request_completed_ack_codes'].apply(lambda x: True in [(np.isnan(i) or i==15) for i in x])
transfers = transfers_raw.copy()
transfers.loc[successful_transfers_bool, "status"] = "INTEGRATED"

# Pending (Sender Error) Code
pending_sender_error_codes=[6,7,10,24,30,23,14,99]
transfers_with_pending_sender_code_bool=transfers['sender_error_code'].isin(pending_sender_error_codes)
transfers_with_pending_with_error_bool=transfers['status']=='PENDING_WITH_ERROR'
transfers_which_need_pending_to_failure_change_bool=transfers_with_pending_sender_code_bool & transfers_with_pending_with_error_bool
transfers.loc[transfers_which_need_pending_to_failure_change_bool,'status']='FAILED'

# Add integrated Late status
eight_days_in_seconds=8*24*60*60
transfers_after_sla_bool=transfers['sla_duration']>eight_days_in_seconds
transfers_with_integrated_bool=transfers['status']=='INTEGRATED'
transfers_integrated_late_bool=transfers_after_sla_bool & transfers_with_integrated_bool
transfers.loc[transfers_integrated_late_bool,'status']='INTEGRATED LATE'

twenty_eight_days_in_seconds=28*24*60*60
transfers_after_month_bool=transfers['sla_duration']>twenty_eight_days_in_seconds
transfers_pending_at_month_bool=transfers_after_month_bool & transfers_integrated_late_bool
transfers.loc[transfers_pending_at_month_bool,'status']='PENDING'
transfers_with_early_error_bool=(~transfers.loc[:,'sender_error_code'].isna()) |(~transfers.loc[:,'intermediate_error_codes'].apply(len)>0)
transfers.loc[transfers_with_early_error_bool & transfers_pending_at_month_bool,'status']='PENDING_WITH_ERROR'

# Supplier name mapping
supplier_renaming = {
    "EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)":"EMIS",
    "IN PRACTICE SYSTEMS LTD":"Vision",
    "MICROTEST LTD":"Microtest",
    "THE PHOENIX PARTNERSHIP":"TPP",
    None: "Unknown"
}

asid_lookup_file = "s3://prm-gp2gp-data-sandbox-dev/asid-lookup/asidLookup-Mar-2021.csv.gz"
asid_lookup = pd.read_csv(asid_lookup_file)
lookup = asid_lookup[["ASID", "MName", "NACS","OrgName"]]

transfers = transfers.merge(lookup, left_on='requesting_practice_asid',right_on='ASID',how='left')
transfers = transfers.rename({'MName': 'requesting_supplier', 'ASID': 'requesting_supplier_asid', 'NACS': 'requesting_ods_code'}, axis=1)
transfers = transfers.merge(lookup, left_on='sending_practice_asid',right_on='ASID',how='left')
transfers = transfers.rename({'MName': 'sending_supplier', 'ASID': 'sending_supplier_asid', 'NACS': 'sending_ods_code'}, axis=1)

transfers["sending_supplier"] = transfers["sending_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())
transfers["requesting_supplier"] = transfers["requesting_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())

In [169]:
transfers_reduced_columns = transfers[["requesting_practice_asid","requesting_supplier","sending_supplier", "status"]].copy()
is_pending_transfers = transfers["status"] == "PENDING"
transfers_reduced_columns["is_pending"] = is_pending_transfers
transfers_reduced_columns = transfers_reduced_columns.drop("status", axis=1)
transfers_reduced_columns

,requesting_practice_asid,requesting_supplier,sending_supplier,is_pending
0,323917613040,EMIS,EMIS,False
1,792911523019,EMIS,EMIS,False
2,981416634047,EMIS,EMIS,False
3,310097028016,EMIS,EMIS,False
4,200000001906,TPP,EMIS,False
...,...,...,...,...
1343229,994133653042,EMIS,TPP,True
1343230,345196753049,EMIS,EMIS,False
1343231,200000001410,EMIS,EMIS,True
1343232,200000000572,EMIS,EMIS,False


In [170]:
pending_transfers_supplier_pathways_pivot = pd.pivot_table(transfers_reduced_columns, index=["requesting_supplier", "requesting_practice_asid"], columns="sending_supplier", values="is_pending", aggfunc="sum").fillna(0)

suppliers_to_investigate = ["EMIS", "TPP", "Vision"]
pending_transfers_as_list = [pending_transfers_supplier_pathways_pivot.loc[supplier].corr().stack().rename(supplier) for supplier in suppliers_to_investigate]
pending_transfers_volume_between_suppliers_correlation = pd.concat(pending_transfers_as_list, axis=1)
pending_transfers_volume_between_suppliers_correlation.loc[[("EMIS", "TPP"), ("EMIS", "Vision"), ("TPP", "Vision")]].round(2)

EMIS   TPP  Vision
sending_supplier sending_supplier                    
EMIS             TPP               0.81 -0.00    0.81
                 Vision            0.09  0.24    0.21
TPP              Vision            0.05  0.02    0.09

In [171]:
pending_transfers_supplier_pathways_pivot = pd.pivot_table(transfers_reduced_columns, index=["requesting_supplier", "requesting_practice_asid"], columns="sending_supplier", values="is_pending", aggfunc="mean").fillna(0)

suppliers_to_investigate = ["EMIS", "TPP", "Vision"]
pending_transfers_as_list = [pending_transfers_supplier_pathways_pivot.loc[supplier].corr().stack().rename(supplier) for supplier in suppliers_to_investigate]
pending_transfers_percentage_between_suppliers_correlation = pd.concat(pending_transfers_as_list, axis=1)
pending_transfers_percentage_between_suppliers_correlation.loc[[("EMIS", "TPP"), ("EMIS", "Vision"), ("TPP", "Vision")]].round(2)

EMIS   TPP  Vision
sending_supplier sending_supplier                    
EMIS             TPP               0.85 -0.01    0.48
                 Vision            0.08  0.06    0.31
TPP              Vision            0.08  0.03    0.17

In [172]:
def create_pending_transfers_with_org_info_for_supplier(supplier,transfers_df):
    pending_transfers_supplier_pathways_pivot = pd.pivot_table(transfers_df, index=["requesting_supplier", "requesting_practice_asid"], columns="sending_supplier", values="is_pending", aggfunc="sum").fillna(0)
    pending_transfers_for_supplier_pathways = pending_transfers_supplier_pathways_pivot.loc[supplier].copy().astype(int)
    pending_transfers_for_supplier_pathways = pending_transfers_for_supplier_pathways.loc[:, ["EMIS","TPP","Vision","Microtest","Unknown"]]
    pending_transfers_for_supplier_pathways.insert(0, "Total Pending Transfers", pending_transfers_for_supplier_pathways.sum(axis=1))
    
    pending_transfers_supplier_pathways_pivot_percentage = pd.pivot_table(transfers_df, index=["requesting_supplier", "requesting_practice_asid"], columns="sending_supplier", values="is_pending", aggfunc="mean").fillna(0)
    pending_transfers_supplier_pathways_percentage = pending_transfers_supplier_pathways_pivot_percentage.loc[supplier].copy().round(4).multiply(100)
    pending_transfers_supplier_pathways_percentage = pending_transfers_supplier_pathways_percentage.loc[:, ["EMIS","TPP","Vision","Microtest","Unknown"]]
    pending_transfers_supplier_pathways_percentage.columns = pending_transfers_supplier_pathways_percentage.columns + " %"
    
    complete_pending_transfers_for_supplier_pathways = pd.concat([pending_transfers_for_supplier_pathways, pending_transfers_supplier_pathways_percentage], axis=1)
    complete_pending_transfers_for_supplier_pathways = complete_pending_transfers_for_supplier_pathways.sort_values(by="Total Pending Transfers", ascending=False)
    complete_pending_transfers_for_supplier_pathways = asid_lookup[["ASID", "PostCode", "OrgName"]].merge(complete_pending_transfers_for_supplier_pathways, right_index=True, left_on="ASID", how="right")
    return complete_pending_transfers_for_supplier_pathways

In [173]:
emis_pending_transfers_with_org_info = create_pending_transfers_with_org_info_for_supplier("EMIS",transfers_reduced_columns)
emis_pending_transfers_with_org_info.head(20)

,ASID,PostCode,OrgName,Total Pending Transfers,EMIS,TPP,Vision,Microtest,Unknown,EMIS %,TPP %,Vision %,Microtest %,Unknown %
33038,615075629043,NR4 7TJ,UEA MEDICAL CENTRE,2043,917,1079,36,0,11,82.09,83.97,90.00,0.0,64.71
11200,200000010928,ME17 4PL,GREENSANDS,755,5,0,750,0,0,1.42,0.00,65.73,0.0,0.00
33754,669024222046,LA1 1PN,LANCASTER MEDICAL PRACTICE,618,442,145,31,0,0,21.46,18.71,42.47,0.0,0.00
28125,260198413015,ME14 4DS,BEARSTED,368,211,18,139,0,0,63.94,60.00,91.45,0.0,0.00
38052,994133653042,IG2 6UP,GANTS HILL MEDICAL CENTRE,242,201,33,8,0,0,81.05,80.49,100.00,0.0,0.00
31980,540917834041,RG40 1PH,BURMA HILL PRACTICE,229,199,27,3,0,0,87.67,87.10,100.00,0.0,0.00
37143,926102461049,HA9 6PW,HAZELDENE MEDICAL CENTRE,217,139,49,26,0,3,2.86,2.84,28.57,0.0,27.27
3256,200000000897,HA2 7DU,THE RIDGEWAY SURGERY,204,167,36,1,0,0,80.29,76.60,100.00,0.0,0.00
23104,200000023705,SE10 8EX,BURNEY STREET PMS,192,139,32,21,0,0,46.49,48.48,70.00,0.0,0.00
3116,200000000692,CR0 0JA,NEW ADDINGTON GROUP PRACTICE,191,168,18,5,0,0,66.67,69.23,83.33,0.0,0.00


In [174]:
#pd.pivot_table(transfers_reduced_columns, index="requesting_supplier", columns="sending_supplier", values="is_pending", aggfunc="sum")

In [175]:
all_suppliers = [create_pending_transfers_with_org_info_for_supplier(supplier,transfers_reduced_columns) for supplier in suppliers_to_investigate]
all_suppliers = pd.concat(all_suppliers, axis=0).fillna(0)
all_suppliers.sort_values("Total Pending Transfers", ascending=False).head(20)

,ASID,PostCode,OrgName,Total Pending Transfers,EMIS,TPP,Vision,Microtest,Unknown,EMIS %,TPP %,Vision %,Microtest %,Unknown %
33038,615075629043,NR4 7TJ,UEA MEDICAL CENTRE,2043,917,1079,36,0,11,82.09,83.97,90.00,0.0,64.71
11200,200000010928,ME17 4PL,GREENSANDS,755,5,0,750,0,0,1.42,0.00,65.73,0.0,0.00
33754,669024222046,LA1 1PN,LANCASTER MEDICAL PRACTICE,618,442,145,31,0,0,21.46,18.71,42.47,0.0,0.00
28125,260198413015,ME14 4DS,BEARSTED,368,211,18,139,0,0,63.94,60.00,91.45,0.0,0.00
38052,994133653042,IG2 6UP,GANTS HILL MEDICAL CENTRE,242,201,33,8,0,0,81.05,80.49,100.00,0.0,0.00
31980,540917834041,RG40 1PH,BURMA HILL PRACTICE,229,199,27,3,0,0,87.67,87.10,100.00,0.0,0.00
37143,926102461049,HA9 6PW,HAZELDENE MEDICAL CENTRE,217,139,49,26,0,3,2.86,2.84,28.57,0.0,27.27
36736,896286726030,SE9 6ER,EVEREST HEALTH PARTNERSHIP,206,7,1,198,0,0,2.33,2.86,84.62,0.0,0.00
3256,200000000897,HA2 7DU,THE RIDGEWAY SURGERY,204,167,36,1,0,0,80.29,76.60,100.00,0.0,0.00
23104,200000023705,SE10 8EX,BURNEY STREET PMS,192,139,32,21,0,0,46.49,48.48,70.00,0.0,0.00


In [179]:
gants_hill = (transfers["requesting_practice_asid"] == "926102461049") & (transfers["status"] == "PENDING") & (transfers["sending_supplier"] == "Vision")
gants_hill = transfers.loc[gants_hill]

gants_hill

,conversation_id,sla_duration,requesting_practice_asid,sending_practice_asid,sender_error_code,final_error_code,intermediate_error_codes,status,date_requested,date_completed,request_completed_ack_codes,requesting_supplier_asid,requesting_supplier,requesting_ods_code,OrgName_x,sending_supplier_asid,sending_supplier,sending_ods_code,OrgName_y
23086,A7917363-2FBC-4633-9463-5F2BD25D01DB,NaN,926102461049,487540951034,NaN,NaN,[],PENDING,2020-09-29 08:06:13.899,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,487540951034,Vision,G83628,NEW ELTHAM AND BLACKFEN MEDICAL CENTRE
50773,933E4CB5-6151-4F44-B7DE-1240F34F4B3C,NaN,926102461049,727881965014,NaN,NaN,[],PENDING,2020-09-21 15:39:09.885,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,727881965014,Vision,G83663,BRISET CORNER SURGERY
53888,15C4EFF3-A72A-4B3F-9717-EF3FBF1E5DBD,NaN,926102461049,068031573030,NaN,NaN,[],PENDING,2020-09-28 09:29:52.339,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,068031573030,Vision,G83016,ROYAL ARSENAL MEDICAL CENTRE
146618,9F684338-22EF-47B8-A920-A4B5D7F3388C,NaN,926102461049,073481077037,NaN,NaN,[],PENDING,2020-09-16 07:31:04.416,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,073481077037,Vision,P83020,MINDEN FAMILY PRACTICES - DR SAXENA
169131,FD048E80-205F-4F66-B432-6613CA0E4B62,NaN,926102461049,554084368030,NaN,NaN,[],PENDING,2020-09-18 12:29:52.922,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,554084368030,Vision,G83633,CONWAY PMS
176620,42ABF45F-100D-4D6F-A940-D147761726EE,NaN,926102461049,728658052036,NaN,NaN,[],PENDING,2020-09-14 05:24:56.021,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,728658052036,Vision,F82038,DR SZ HAIDER'S PRACTICE
275204,0396C283-FA9D-4A48-8551-94B2445AF46C,NaN,926102461049,071822344017,NaN,NaN,[],PENDING,2020-10-31 09:46:04.584,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,071822344017,Vision,P87027,LANGWORTHY MEDICAL PRACTICE
539565,A72BA811-913B-40C7-9411-BE8A6503222D,NaN,926102461049,896286726030,NaN,NaN,[],PENDING,2020-11-25 10:24:46.946,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,896286726030,Vision,G83027,EVEREST HEALTH PARTNERSHIP
749207,30582160-4D0B-4C4E-9C70-9F08ACEE3B3C,NaN,926102461049,598365232035,NaN,NaN,[],PENDING,2020-12-25 01:14:57.358,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,598365232035,Vision,G83026,TRIVENI PMS
763508,CD84686F-2DDF-4EC8-9340-7715AD7E9ABC,NaN,926102461049,801645630030,NaN,NaN,[],PENDING,2020-12-18 15:07:31.577,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,801645630030,Vision,G83062,SLADE GREEN MEDICAL CTR


In [180]:



# Given the findings in PRMT-1742 - many duplicate EHR errors are misclassified, the below reclassifies the relevant data
successful_transfers_boolean = gants_hill['request_completed_ack_codes'].apply(lambda x: True in [(np.isnan(i) or i==15) for i in x])
gants_hill.loc[successful_transfers_boolean, "status"] = "INTEGRATED"

gants_hill
#gants_hill.loc[successful_transfers_boolean]
#transfers = transfers_raw.copy()
#transfers.loc[successful_transfers_bool, "status"] = "INTEGRATED"

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,conversation_id,sla_duration,requesting_practice_asid,sending_practice_asid,sender_error_code,final_error_code,intermediate_error_codes,status,date_requested,date_completed,request_completed_ack_codes,requesting_supplier_asid,requesting_supplier,requesting_ods_code,OrgName_x,sending_supplier_asid,sending_supplier,sending_ods_code,OrgName_y
23086,A7917363-2FBC-4633-9463-5F2BD25D01DB,NaN,926102461049,487540951034,NaN,NaN,[],PENDING,2020-09-29 08:06:13.899,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,487540951034,Vision,G83628,NEW ELTHAM AND BLACKFEN MEDICAL CENTRE
50773,933E4CB5-6151-4F44-B7DE-1240F34F4B3C,NaN,926102461049,727881965014,NaN,NaN,[],PENDING,2020-09-21 15:39:09.885,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,727881965014,Vision,G83663,BRISET CORNER SURGERY
53888,15C4EFF3-A72A-4B3F-9717-EF3FBF1E5DBD,NaN,926102461049,068031573030,NaN,NaN,[],PENDING,2020-09-28 09:29:52.339,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,068031573030,Vision,G83016,ROYAL ARSENAL MEDICAL CENTRE
146618,9F684338-22EF-47B8-A920-A4B5D7F3388C,NaN,926102461049,073481077037,NaN,NaN,[],PENDING,2020-09-16 07:31:04.416,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,073481077037,Vision,P83020,MINDEN FAMILY PRACTICES - DR SAXENA
169131,FD048E80-205F-4F66-B432-6613CA0E4B62,NaN,926102461049,554084368030,NaN,NaN,[],PENDING,2020-09-18 12:29:52.922,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,554084368030,Vision,G83633,CONWAY PMS
176620,42ABF45F-100D-4D6F-A940-D147761726EE,NaN,926102461049,728658052036,NaN,NaN,[],PENDING,2020-09-14 05:24:56.021,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,728658052036,Vision,F82038,DR SZ HAIDER'S PRACTICE
275204,0396C283-FA9D-4A48-8551-94B2445AF46C,NaN,926102461049,071822344017,NaN,NaN,[],PENDING,2020-10-31 09:46:04.584,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,071822344017,Vision,P87027,LANGWORTHY MEDICAL PRACTICE
539565,A72BA811-913B-40C7-9411-BE8A6503222D,NaN,926102461049,896286726030,NaN,NaN,[],PENDING,2020-11-25 10:24:46.946,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,896286726030,Vision,G83027,EVEREST HEALTH PARTNERSHIP
749207,30582160-4D0B-4C4E-9C70-9F08ACEE3B3C,NaN,926102461049,598365232035,NaN,NaN,[],PENDING,2020-12-25 01:14:57.358,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,598365232035,Vision,G83026,TRIVENI PMS
763508,CD84686F-2DDF-4EC8-9340-7715AD7E9ABC,NaN,926102461049,801645630030,NaN,NaN,[],PENDING,2020-12-18 15:07:31.577,NaT,[],926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE,801645630030,Vision,G83062,SLADE GREEN MEDICAL CTR


In [182]:
transfers[transfers["status"] == "PENDING"]

,conversation_id,sla_duration,requesting_practice_asid,sending_practice_asid,sender_error_code,final_error_code,intermediate_error_codes,status,date_requested,date_completed,request_completed_ack_codes,requesting_supplier_asid,requesting_supplier,requesting_ods_code,OrgName_x,sending_supplier_asid,sending_supplier,sending_ods_code,OrgName_y
39,3A743BDF-9161-42CE-8A06-73A45EAA9E2C,NaN,819979159012,248516836017,NaN,NaN,[],PENDING,2020-09-30 08:09:41.216,NaT,[],819979159012,EMIS,L81081,PEMBROKE ROAD SURGERY,248516836017,Vision,G83651,WOODLANDS SURGERY
75,B7CDD030-0339-11EB-B189-6F70B7E3218C,NaN,825608078010,736940363012,NaN,NaN,[],PENDING,2020-09-30 16:26:40.039,NaT,[],825608078010,TPP,B86025,HYDE PARK SURGERY,736940363012,EMIS,B86110,LEEDS STUDENT MEDICAL PRACTICE
134,785B5D30-032C-11EB-AF2A-919014107559,NaN,200000000245,200000015375,NaN,NaN,[],PENDING,2020-09-30 14:51:50.392,NaT,[],200000000245,TPP,Y04942,UNIVERSITY HEALTH CENTRE,200000015375,EMIS,G82002,ST JAMES' SURGERY
144,0EA6BD35-5363-4976-AD32-DE24A0D12151,2591060.0,865992626016,005843584045,NaN,NaN,[],PENDING,2020-09-30 14:51:38.637,2020-10-30 14:36:10.137,[nan],865992626016,EMIS,F84117,QUEENSBRIDGE GROUP PRACTICE,005843584045,EMIS,F83632,QUEENS CRESCENT PRACTICE
170,1ED4849A-0811-4059-AB34-BC7384F897BA,NaN,457645785017,586797122015,NaN,NaN,[],PENDING,2020-09-30 15:19:42.810,NaT,[],457645785017,EMIS,G82041,GROSVENOR & ST JAMES MEDICAL CENTRE,586797122015,TPP,Y02807,THURROCK HEALTH CENTRE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343219,DE4F4B53-26DE-4784-B25F-F40A4AE546D5,NaN,200000001210,175596650036,NaN,NaN,[],PENDING,2021-02-01 11:15:09.645,NaT,[],200000001210,EMIS,P81143,WHITTLE SURGERY,175596650036,Vision,P87020,ST ANDREWS MEDICAL CENTRE
1343225,EC50AA08-F7BE-4F2C-9FE8-834CD99D3ABD,NaN,401437889041,926102461049,NaN,NaN,[],PENDING,2021-02-01 10:52:29.613,NaT,[],401437889041,EMIS,G85102,THE VAUXHALL SURGERY,926102461049,EMIS,E84066,HAZELDENE MEDICAL CENTRE
1343227,8D460AD0-646E-11EB-9067-4B9671D6A261,NaN,359793262049,202393302015,NaN,NaN,[],PENDING,2021-02-01 09:19:14.884,NaT,[],359793262049,TPP,C88034,NETHERGREEN SURGERY,202393302015,EMIS,G85711,DEPTFORD SURGERY
1343229,75173BD9-37A6-4E15-856E-F173317D01E6,NaN,994133653042,950140416013,NaN,NaN,[],PENDING,2021-02-01 10:38:41.327,NaT,[],994133653042,EMIS,F86008,GANTS HILL MEDICAL CENTRE,950140416013,TPP,F81110,TILBURY HEALTH CENTRE
